In [1]:
import requests
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
import datetime

In [ ]:
'''
数据输入：交易日DataFrame，列名Date，格式为日期
'''

In [2]:
DateSeries = pd.read_excel('C:/Users/Thinkpad/Desktop/TradeDate.xlsx')['Date']
DateDf = DateSeries.apply(lambda x: x.strftime('%Y%m%d')).to_frame()
DateList = DateDf['Date'].tolist()
OutPutPath = 'C:/Users/Thinkpad/Desktop/'

In [ ]:
'''
单种类爬虫：IF、IC、IH
'''

In [7]:
kind = 'IH'

# LongVarV_sum_SaveList = []
# ShortVarV_sum_SaveList = []

# DB1 = pd.DataFrame()
# DB2 = pd.DataFrame()
# DB3 = pd.DataFrame()

cnt = 0
for TradeDate in DateList:
    
    # 非第一次才跑！增量写入
    if TradeDate in DB1['日期'].unique().tolist():
        print(TradeDate+' '+kind+' Complete',end='\r')
        continue
    
    Y_m = TradeDate[:6]
    d = TradeDate[6:]
    
    URL = 'http://www.cffex.com.cn/sj/ccpm/'+Y_m+'/'+d+'/'+kind+'.xml'
    r = requests.get(url=URL)
    
    WebText = BeautifulSoup(r.text,"lxml")
    r.close()
    
    # 成交量排名
    reForTradeV = '<data text=".*?                        " value="0">\n    <instrumentid>\n     (.*?)\n    </instrumentid>\n    <tradingday>\n     (.*?)\n    </tradingday>\n    <datatypeid>\n     .*?\n    </datatypeid>\n    <rank>\n     (.*?)\n    </rank>\n    <shortname>\n     (.*?)\n    </shortname>\n    <volume>\n     (.*?)\n    </volume>\n    <varvolume>\n     (.*?)\n    </varvolume>\n    <partyid>\n     .*?\n    </partyid>\n    <productid>\n     .*?\n    </productid>\n   </data>\n'
    # 多单排名
    reForLong = '<data text=".*?                        " value="1">\n    <instrumentid>\n     (.*?)\n    </instrumentid>\n    <tradingday>\n     (.*?)\n    </tradingday>\n    <datatypeid>\n     .*?\n    </datatypeid>\n    <rank>\n     (.*?)\n    </rank>\n    <shortname>\n     (.*?)\n    </shortname>\n    <volume>\n     (.*?)\n    </volume>\n    <varvolume>\n     (.*?)\n    </varvolume>\n    <partyid>\n     .*?\n    </partyid>\n    <productid>\n     .*?\n    </productid>\n   </data>\n'
    # 空单排名
    reForShort = '<data text=".*?                        " value="2">\n    <instrumentid>\n     (.*?)\n    </instrumentid>\n    <tradingday>\n     (.*?)\n    </tradingday>\n    <datatypeid>\n     .*?\n    </datatypeid>\n    <rank>\n     (.*?)\n    </rank>\n    <shortname>\n     (.*?)\n    </shortname>\n    <volume>\n     (.*?)\n    </volume>\n    <varvolume>\n     (.*?)\n    </varvolume>\n    <partyid>\n     .*?\n    </partyid>\n    <productid>\n     .*?\n    </productid>\n   </data>\n'
    
    TradeVData = re.findall(reForTradeV,WebText.prettify())
    
    LongData = re.findall(reForLong,WebText.prettify())
#     LongVarV = re.findall(reForLong,WebText.prettify())
#     LongVarV_int = [int(i) for i in LongVarV]
    
    ShortData = re.findall(reForShort,WebText.prettify())
#     ShortVarV = re.findall(reForShort,WebText.prettify())
#     ShortVarV_int = [int(i) for i in ShortVarV]
    
#     LongVarV_sum_SaveList.append(np.sum(LongVarV_int))
#     ShortVarV_sum_SaveList.append(np.sum(ShortVarV_int))

    DB1_ = pd.DataFrame(TradeVData,columns=['合约名称','日期','排名','公司','量','变化量'])
    DB2_ = pd.DataFrame(LongData,columns=['合约名称','日期','排名','公司','量','变化量'])
    DB3_ = pd.DataFrame(ShortData,columns=['合约名称','日期','排名','公司','量','变化量'])
    
    DB1 = pd.concat([DB1,DB1_],axis=0)
    DB2 = pd.concat([DB2,DB2_],axis=0)
    DB3 = pd.concat([DB3,DB3_],axis=0)
    
    cnt += 1
    print(TradeDate+' '+kind+' Complete',end='\r')

In [8]:
DB1.to_excel(OutPutPath+kind+'成交量排名'+'蜘蛛网策略数据更新至'+DateList[-1]+'.xlsx')
DB2.to_excel(OutPutPath+kind+'持买单量排名'+'蜘蛛网策略数据更新至'+DateList[-1]+'.xlsx')
DB3.to_excel(OutPutPath+kind+'持卖单量排名'+'蜘蛛网策略数据更新至'+DateList[-1]+'.xlsx')

In [ ]:
'''
单种类结果输出：路径+种类+蜘蛛网策略数据更新至+最新日期
'''

In [142]:
SaveDf = pd.DataFrame({'日期':DateSeries.tolist(),'多单持仓增量':LongVarV_sum_SaveList,'空单持仓增量':ShortVarV_sum_SaveList})
SaveDf.to_excel(OutPutPath+kind+'蜘蛛网策略数据更新至'+DateList[-1]+'.xlsx')

In [ ]:
'''
单PY文件形式
'''

In [143]:
import requests
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
import datetime

def TradeDateProcess(excelpath):
    
    TradeDateExcelPath = excelpath
    
    DateSeries = pd.read_excel(TradeDateExcelPath)['Date']
    DateDf = DateSeries.apply(lambda x: x.strftime('%Y%m%d')).to_frame()
    DateList = DateDf['Date'].tolist()
    
    return DateList
    
def ReForLongShort(text):
    
    WebText = text
    
    reForLong = '<data text=".*?                        " value="1">\n    <instrumentid>\n     .*?\n    </instrumentid>\n    <tradingday>\n     .*?\n    </tradingday>\n    <datatypeid>\n     .*?\n    </datatypeid>\n    <rank>\n     .*?\n    </rank>\n    <shortname>\n     .*?\n    </shortname>\n    <volume>\n     .*?\n    </volume>\n    <varvolume>\n     (.*?)\n    </varvolume>\n    <partyid>\n     .*?\n    </partyid>\n    <productid>\n     .*?\n    </productid>\n   </data>\n'
    reForShort = '<data text=".*?                        " value="2">\n    <instrumentid>\n     .*?\n    </instrumentid>\n    <tradingday>\n     .*?\n    </tradingday>\n    <datatypeid>\n     .*?\n    </datatypeid>\n    <rank>\n     .*?\n    </rank>\n    <shortname>\n     .*?\n    </shortname>\n    <volume>\n     .*?\n    </volume>\n    <varvolume>\n     (.*?)\n    </varvolume>\n    <partyid>\n     .*?\n    </partyid>\n    <productid>\n     .*?\n    </productid>\n   </data>\n'

    LongVarV = re.findall(reForLong,WebText.prettify())
    LongVarV_int = [int(i) for i in LongVarV]

    ShortVarV = re.findall(reForShort,WebText.prettify())
    ShortVarV_int = [int(i) for i in ShortVarV]

    LongVarV_sum_SaveList.append(np.sum(LongVarV_int))
    ShortVarV_sum_SaveList.append(np.sum(ShortVarV_int))
    
    return LongVarV_sum_SaveList, ShortVarV_sum_SaveList


def GetQuery(kind,DateList):
    
    kind, DateList = kind, DateList

    LongVarV_sum_SaveList = []
    ShortVarV_sum_SaveList = []

    for TradeDate in DateList:
        Y_m = TradeDate[:6]
        d = TradeDate[6:]

        URL = 'http://www.cffex.com.cn/sj/ccpm/'+Y_m+'/'+d+'/'+kind+'.xml'
        r = requests.get(url=URL)

        WebText = BeautifulSoup(r.text,"lxml")
        r.close()
        
        print(TradeDate+' '+kind+' Complete',end='\r')
        
    return ReForLongShort(WebText)
    
def OutPut(longdatalist,shortdatalist,OutPutPath):
    
    LongVarV_sum_SaveList, ShortVarV_sum_SaveList = longdatalist, shortdatalist
    
    SaveDf = pd.DataFrame({'日期':DateList,'多单持仓增量':LongVarV_sum_SaveList,'空单持仓增量':ShortVarV_sum_SaveList})
    SaveDf.to_excel(OutPutPath+kind+'蜘蛛网策略数据更新至'+DateList[-1]+'.xlsx')
    
def Run(DateList,kindList,OutPutPath):
    
    for kind in kindList:
        LongVarV_sum_SaveList, ShortVarV_sum_SaveList = GetQuery(kind,DateList)
        OutPut(longdatalist,shortdatalist,OutPutPath)
    
if __name__ == '__main__':
    
    # 日期数据，原始文件为excel，列名Date，数据格式为日期形式
    TradeDateExcelPath = 'C:/Users/Thinkpad/Desktop/TradeDate.xlsx'
    
    # 爬取种类
    kindList = ['IF','IC','IH']
    
    # 数据输出路径
    OutPutPath = 'C:/Users/Thinkpad/Desktop/'
    
    # 日期数据处理
    DateList = TradeDateProcess(TradeDateExcelPath)
    
    # 爬取数据
    Run(DateList,kindList,OutPutPath)

KeyboardInterrupt: 